In [ ]:
import requests
import csv
from ltr.data import CorpusApi, Config
import numpy as np
import json

In [ ]:
queries = CorpusApi.getValidationQueriesAsDict()

In [ ]:
from nltk.tokenize import word_tokenize
import re

modelName = 'ranksvm-0.001-squared_hinge'

for i, key in enumerate(list(queries.keys())[:]):
    try:
        # get query and clean the query string
        query = queries[key]
        query = re.sub('[^\w\s]', '', query)
        
        # prepare for ltr
        queryTokens = word_tokenize(query)
        tcqtValues = ', '.join([f"if(termfreq(title_classic,'{t}'),1,0)" for t in queryTokens])
        hcqtValues = ', '.join([f"if(termfreq(headings_classic,'{t}'),1,0)" for t in queryTokens])
        bcqtValues = ', '.join([f"if(termfreq(body_classic,'{t}'),1,0)" for t in queryTokens])
        dcqtValues = ', '.join([f"if(termfreq(_copy_all_classic_,'{t}'),1,0)" for t in queryTokens])
        ttfValues = ', '.join([f"tf(title_classic,'{t}')" for t in queryTokens])
        htfValues = ', '.join([f"tf(headings_classic,'{t}')" for t in queryTokens])
        btfValues = ', '.join([f"tf(body_classic,'{t}')" for t in queryTokens])
        dtfValues = ', '.join([f"tf(_copy_all_classic_,'{t}')" for t in queryTokens])
        tidfValues = ', '.join([f"idf(title_classic,'{t}')" for t in queryTokens])
        hidfValues = ', '.join([f"idf(headings_classic,'{t}')" for t in queryTokens])
        bidfValues = ', '.join([f"idf(body_classic,'{t}')" for t in queryTokens])
        didfValues = ', '.join([f"idf(_copy_all_classic_,'{t}')" for t in queryTokens])
        ttfidfValues = ', '.join([f"product(tf(title_classic,'{t}'),idf(title_classic,'{t}'))" for t in queryTokens])
        htfidfValues = ', '.join([f"product(tf(headings_classic,'{t}'),idf(headings_classic,'{t}'))" for t in queryTokens])
        btfidfValues = ', '.join([f"product(tf(body_classic,'{t}'),idf(body_classic,'{t}'))" for t in queryTokens])
        dtfidfValues = ', '.join([f"product(tf(_copy_all_classic_,'{t}'),idf(_copy_all_classic_,'{t}'))" for t in queryTokens])
        
        

        # get result from solr
        request = {
            "fields": "id, score",
            "limit": 500,
            "params": {
                "rq": "{!ltr reRankDocs=500 " + 
                    f"""
                  efi.keywords=\"{query}\" 
                  efi.tcqt_values=\"  {tcqtValues}
                  \" 
                  efi.hcqt_values=\"  {hcqtValues}
                  \"
                  efi.bcqt_values=\"  {bcqtValues}
                  \"
                  efi.dcqt_values=\"  {dcqtValues}
                  \"
                  efi.query_terms_length={len(queryTokens)}
                  efi.ttf_values=\"  {ttfValues}
                  \" 
                  efi.htf_values=\"  {htfValues}
                  \" 
                  efi.btf_values=\"  {btfValues}
                  \" 
                  efi.dtf_values=\"  {dtfValues}
                  \" 
                  efi.tidf_values=\"  {tidfValues}
                  \" 
                  efi.hidf_values=\"  {hidfValues}
                  \" 
                  efi.bidf_values=\"  {bidfValues}
                  \" 
                  efi.didf_values=\"  {didfValues}
                  \" 
                  efi.ttfidf_values=\"  {ttfidfValues}
                  \" 
                  efi.htfidf_values=\"  {htfidfValues}
                  \" 
                  efi.btfidf_values=\"  {btfidfValues}
                  \" 
                  efi.dtfidf_values=\"  {dtfidfValues}
                  \" 
          """
                +"model="+modelName+"}",
                "qf": "_copy_all_",
                "defType": "dismax",
                "q": query
            }
        }
        
        
        response = requests.post(f'http://localhost:8983/solr/thesis-ltr/select', json=request)
        
        queryResult = [doc for doc in response.json()["response"]["docs"]]
        
        
        # write resultfile for trec eval
        with open(f'submission/ranksvm/evaluation/{modelName}.results', 'a') as file:
            for j, doc in enumerate(queryResult):
                line = " ".join([str(key), "Q0", str(doc['id']), str(j + 1), str(doc['score']), "STANDARD"])
                file.write(line)
                file.write('\n')
        
        print(f'Processed queries: {i + 1}/{len(list(queries.keys()))}')
    except KeyError as err:
        # ignore the queries that are not in the validation queries list
        print(err)
        print(queryTokens)
        print(response.json())
        pass